In [1]:
# import matplotlib.pyplot as plt
import numpy as np
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.linear_model import LinearRegression
import csv
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.linear_model import LogisticRegression
# from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from datetime import datetime, date, time,timedelta
import pickle
import sqlite3
from sqlite3 import Error
import datetime

In [ ]:
import socketio
import json
#import decision_tree

#ip = 'http://35.246.29.217:65080/' 
#ip = 'http://146.169.173.109:65080/' 
#ip = 'http://localhost:65080'
#ip = 'http://146.169.161.84:65080'
ip = 'http://146.169.171.19:65080'
sio = socketio.Client()

@sio.on('connect')
def on_connect():
    print('connection established')

@sio.on('newTime')
def on_message(data):
    #data = json.loads(data['message'])
    hour = data['hour']
    minute = data['minute']
    second = data['second']
    # call function to increase level
    t = DecisionTree(hour,minute)
    level = t.analysis()
    print('predicted level: ', level.tolist())
    sio.emit('newAlertLevel', level[0]-0.)

@sio.on('disconnect')
def on_disconnect():
    print('disconnected from server')

sio.connect(ip)
sio.wait()
#sio.sleep(1)

connection established
predicted level:  [3.0]
predicted level:  [3.0]
predicted level:  [3.0]
predicted level:  [3.0]


In [2]:
class DecisionTree():
    def __init__(self, hour,minute):
        self.hour = hour
        self.minute = minute
        
    def create_connection(self,db_file):
        try:
            conn = sqlite3.connect(db_file)
            return conn
        except Error as e:
            print(e)
        return None
    
    def get_db_Time(self,conn):
        cur = conn.cursor()
        cur.execute("SELECT Wake, Breakfast, Lunch, Dinner, Sleep From questions WHERE Name = 'Louis Kueh'")
        rows = cur.fetchall()
        return rows

    def get_db_daily_time(self,conn):
        cur = conn.cursor()
        #cur.execute("SELECT Timestamp From timeTaken")
        cur.execute("SELECT Timestamp From timeTaken WHERE BoxNo = 1")
        rows = cur.fetchall()
        return rows
    
    def get_init(self):
        database = './storage.db'
        conn = self.create_connection(database)
        with conn:
            init_time = self.get_db_daily_time(conn)
        
        actual_time = [x[0][3:24] for x in init_time]
        
        t = (actual_time[0].replace('Mar','03').replace(' ','/'))[12:]

        year = '19'
        month = '01'
        day = '01'
        t = year+'/'+month+'/'+day+ ' '+t
        actual_time_convert = datetime.datetime.strptime(t, "%y/%m/%d %H:%M:%S")
        #print('actual_time_convert', actual_time_convert)
        return actual_time_convert
    
    def analysis(self):
        year = '19'
        month = '01'
        day = '01'
        time = year+'/'+month+'/'+day+ ' ' + self.hour + ':' + self.minute
        test_time = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")
        #print('test_time ', test_time)
        init_time = self.get_init() #get initial setting time to take the medcine
        diff = abs(test_time-init_time) # find difference between actual and pre-set
        #print(diff)
        x_in = np.array([(diff.total_seconds()/60)]) # convert to minutes
        
        # Load model for testing
        filename = 'notification_model.sav'
        model = pickle.load(open(filename, 'rb'))
        y_predict = model.predict(x_in.reshape(-1,1))
        
        return y_predict # notification level

# t = DecisionTree('9','55')
# level = t.analysis()
# print(level)